<a href="https://colab.research.google.com/github/ilkayyagizgur/Fake_News_Training/blob/main/Fake_News_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Import Required Libraries


In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [ ]:
!pip install transformers
!pip install datasets
!pip install numpy
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:0

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pyarrow as pa
import pandas as pd
import numpy as np
import evaluate
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Model Settings


In [ ]:
choosen_model ='dbmdz/bert-base-turkish-cased'
number_labels = 2

In [ ]:
dataset_location ='/content/drive/MyDrive/Bitirme/LastDatasetWithAllAug.csv'

In [ ]:
trainer_output_dir = "/content/drive/MyDrive/Berturk-cased-model"

In [ ]:
saving_location = '/content/drive/MyDrive/Bert/Models/Berturk-cased-model'
loading_location = saving_location

# Choose model and Download


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(choosen_model)
model = AutoModelForSequenceClassification.from_pretrained(
    choosen_model,
    num_labels=number_labels
)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

In [ ]:
metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

# Defining Required Functions


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=predictions, references=labels)

In [ ]:
def process_data(row):

    text = row['Orj_Text']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text,
                          padding="max_length",
                          truncation=True,
                          max_length=128)

    label = 0
    if row['label'] == 1:
        label += 1

    encodings['labels'] = label
    encodings['text'] = text

    return encodings

# Import and Procces Dataset



In [ ]:
dataset = pd.read_csv(dataset_location)
dataset.head()

,Unnamed: 0,Orj_Text,label
0,0,"ABD Dışişleri Bakanlığı Sözcüsü Toner, ""NATO m...",1
1,1,"Almanya Başbakanı Merkel, İstanbul'daki terör ...",1
2,2,"BERLİN\nGeçen yılı terör korkusu, cumhurbaşkan...",1
3,3,"SAO PAULO\nBrezilya'da, yeni yıl gecesi eski e...",1
4,4,RAMALLAH\nFilistin Devlet Başkanı Mahmud Abbas...,1


In [ ]:
processed_dataset = []

for i in range(len(dataset[:20037])):
    processed_dataset.append(process_data(dataset.iloc[i]))

In [ ]:
processed_dataset[0]

{'input_ids': [2, 3077, 6405, 4184, 15029, 19286, 1972, 16, 6, 9194, 16151, 2177, 2319, 11, 13539, 28457, 2362, 16390, 3844, 8138, 18, 2123, 2784, 20346, 16, 2701, 2319, 9778, 3146, 28457, 2362, 14632, 2680, 2638, 8423, 24512, 12752, 2055, 2077, 18, 6, 2859, 18, 59, 4095, 1052, 29047, 17271, 1037, 3077, 6405, 4184, 15029, 7315, 19286, 1972, 16, 2673, 11, 2054, 4418, 2665, 4024, 27880, 3609, 8139, 3716, 2178, 13161, 1985, 3075, 6857, 3284, 18106, 2073, 5034, 18, 31932, 19286, 1972, 16, 5236, 1996, 3254, 5247, 2673, 11, 2054, 4418, 2665, 4024, 17997, 2538, 14705, 5139, 7340, 18, 2178, 14797, 1992, 16067, 5388, 9465, 2208, 21884, 22553, 19318, 1007, 2353, 15703, 19286, 1972, 16, 6955, 5920, 30, 6, 26706, 3716, 2178, 13161, 1985, 3075, 11600, 3844, 10099, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
new_df = pd.DataFrame(processed_dataset)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=42
)

In [ ]:
train_dataset = Dataset(pa.Table.from_pandas(train_df))
valid_dataset = Dataset(pa.Table.from_pandas(valid_df))

# Setting Up Trainer

In [ ]:
training_args = TrainingArguments(output_dir=trainer_output_dir, evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train and Evaluate


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.084100,0.042112,0.990029,0.990244,0.983845,0.996727
2,0.018500,0.007741,0.998754,0.998771,1.000000,0.997545
3,0.005000,0.012650,0.997507,0.997549,0.995922,0.999182


TrainOutput(global_step=3612, training_loss=0.060426499888482414, metrics={'train_runtime': 383.6528, 'train_samples_per_second': 75.279, 'train_steps_per_second': 9.415, 'total_flos': 1899727597463040.0, 'train_loss': 0.060426499888482414, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.012650213204324245,
 'eval_accuracy': 0.997507270461155,
 'eval_f1': 0.997549019607843,
 'eval_precision': 0.9959216965742251,
 'eval_recall': 0.9991816693944353,
 'eval_runtime': 6.6488,
 'eval_samples_per_second': 362.021,
 'eval_steps_per_second': 45.271,
 'epoch': 3.0}

# Save Model


In [ ]:
model.save_pretrained(saving_location)

In [ ]:
tokenizer.save_pretrained(saving_location, legacy_format=False)

('/content/drive/MyDrive/Bert/Models/Berturk-cased-model/tokenizer_config.json',
 '/content/drive/MyDrive/Bert/Models/Berturk-cased-model/special_tokens_map.json',
 '/content/drive/MyDrive/Bert/Models/Berturk-cased-model/tokenizer.json')

# Load Model


In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained(loading_location)

new_tokenizer = AutoTokenizer.from_pretrained(choosen_model)

loading configuration file /content/drive/MyDrive/Bert/Models/Berturk-cased/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/Bert/Models/Berturk-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file /content/drive/MyDrive/Bert/Models/Berturk-cased/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClas

# Prediction


In [ ]:
def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(loaded_model.device) for k,v in encoding.items()}

    outputs = loaded_model(**encoding)

    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)
    
    return label
      

In [ ]:
get_prediction('Son dakika: Savunma Sanayisinden çok önemli proje! Ve imzalar atıldı')

1

# Accuracy

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
prediction  = []
referance = []
print(len(valid_df))

for i in range(len(valid_df)):
  var = valid_df.iloc[i]["labels"]
  vartwo = get_prediction(valid_df.iloc[i]["text"])
  print(i)

  if var == 0 :
    varthree = 0
  else:
    varthree = 1

  referance.append(varthree)
  prediction.append(vartwo)



accuracy.add_batch(references=referance , predictions=prediction)

accuracy.compute()


In [ ]:
metrics.add_batch(references=referance , predictions=prediction)

metrics.compute()


In [ ]:
for i in range(len(prediction)):
  print(prediction[i],referance[i])

# Azure Back Translation


In [ ]:
# Add your key and endpoint
key = ""
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "westeurope"

path = '/translate'
constructed_url = endpoint + path

In [ ]:
params = {
    'api-version': '3.0',
    'from': 'tr',
    'to': 'en'
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [ ]:
request = requests.post(constructed_url, params=params, headers=headers, json=body)
response = request.json() 
print(  json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

In [ ]:
def TranslateText(text):
  text = text.replace("\n"," ")
  body = [{'text':text}]
  request = requests.post(constructed_url, params=params, headers=headers, json=body)
  response = request.json() 
  return response[0]["translations"][0]["text"]

# Data Augmentation


## Non model augmentations


In [ ]:
sub_char_by_key = nac.KeyboardAug(aug_word_min=10, aug_word_max=30)

In [ ]:
swap_char = nac.RandomCharAug(action="swap", aug_word_min=10, aug_word_max=30)

In [ ]:
delete_char = nac.RandomCharAug(action="delete", aug_word_min=10, aug_word_max=30)

In [ ]:
delete_word = naw.RandomWordAug(aug_min = 10, aug_max = 30)

In [ ]:
delete_set_words = naw.RandomWordAug(action='crop', aug_min = 10, aug_max = 30)

In [ ]:
split_word = naw.SplitAug(aug_min = 10, aug_max = 30)

In [ ]:
swap_words = naw.RandomWordAug(action="swap",aug_min = 10, aug_max = 30)

In [ ]:
def augmentation(df,augmentationModel):
  fake_text=df["Orj_Text"]
  fake_text=fake_text.str.replace("\n", " ")
  texts = list(fake_text)
  aug_list=[]
  for i in range(len(texts)):
    if len(texts[i].split()) <= 10 and augmentationModel == delete_set_words :
      dsw_short= naw.RandomWordAug(action='crop', aug_min = 2)
      augmented_text = dsw_short.augment(texts[i])
    else:
      augmented_text = augmentationModel.augment(texts[i])
    aug_list.append(augmented_text)
    print("Augmented Text:")
    print(augmented_text)
  return aug_list

## Augmentations With Models


In [ ]:
insert_berturk = naw.ContextualWordEmbsAug(
      model_path='dbmdz/bert-base-turkish-cased', action="insert", aug_min = 10, aug_max = 30)
insert_convberturk = naw.ContextualWordEmbsAug(
      model_path='dbmdz/convbert-base-turkish-cased', action="insert", aug_min = 10, aug_max = 30)

augmentations=[insert_Berturk,insert_Convberturk]

In [ ]:
substitute_berturk = naw.ContextualWordEmbsAug(
      model_path='dbmdz/bert-base-turkish-cased', action="substitute", aug_min = 10, aug_max = 30)
substitute_convberturk = naw.ContextualWordEmbsAug(
      model_path='dbmdz/convbert-base-turkish-cased', action="substitute", aug_min = 10, aug_max = 30)

augmentations=[substitute_berturk,substitute_convberturk]

In [ ]:
def augmentation(df):
  fake_text=df["Orj_Text"]
  fake_text=fake_text.str.replace("\n", " ")
  texts = list(fake_text)
  aug_list=[]
  for j in range(len(augmentations)):
    print(augmentations[j])
    for i in range(len(texts)):
      augmented_text = augmentations[j].augment(texts[i])
      aug_list.append(augmented_text)
      print("Augmented Text:")
      print(augmented_text)
  return aug_list

# Stop Word Removal


In [ ]:
#nltk.download()

In [ ]:

example_sent = """This is a sample sentence,
                  showing off the stop words filtration."""
  
stop_words = set(stopwords.words('english'))
  
word_tokens = word_tokenize(example_sent)
# converts the words in word_tokens to lower case and then checks whether 
#they are present in stop_words or not
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []
  
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
  
print(word_tokens)
print(filtered_sentence)

In [ ]:
def remove_stop_words (text):
  word_tokens = word_tokenize(text)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  result = ""
  for w in word_tokens:
    if w not in stop_words:
        result += w
        result +=  " "
  return result 